In [51]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import csv


# Get the list of NASDAQ companies
nasdaq_tickers = pd.read_csv('nasdaq_screener.csv')
#nasdaq_tickers= pd.read_csv('https://datahub.io/core/nasdaq-listings/r/nasdaq-listed-symbols.csv')
tickers_total = nasdaq_tickers['Symbol'].tolist()
tickers = tickers_total[10:13]

# Prepare the CSV file
header_written = False
csv_file = 'nasdaq_financial_data.csv'


# Helper function to get the stock price on a specific date
def get_stock_price(ticker, date):
    for i in range(7):        
        try:
            adjusted_date = date + timedelta(days=i)
            stock_data = yf.download(ticker, start=adjusted_date, end=adjusted_date + timedelta(days=1), progress=False)
            if not stock_data.empty:
                return stock_data['Close'].iloc[0]
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")



# Function that flattens the data of a dataframe concatenate the columns and put everything in a row
def flatten_data(df, ticker):    
    
    df_u = df.unstack().reset_index()
    df_u['Ticker'] = df_u.apply(lambda row: str(row['level_0'].year) + ' ' + row['level_1'], axis=1)
    df_u.set_index(keys = 'Ticker')
    df_ordered = df_u.drop(columns = ['level_0','level_1'])
    df_transposed = df_ordered.set_index(keys='Ticker').T
    df_transposed['Ticker'] = ticker
    df_transposed.set_index(keys='Ticker',inplace = True)  
    
    return df_transposed




In [52]:
#Compute the stock value ratio (item_stock%/sp500_stock%) in a year

def get_stock_value_ratio(date, ticker):
    for i in range(7):
        print(i)
        try:
            # Adjust date by i days
            adjusted_date = date + timedelta(days=i)
            stock_data = get_stock_price(ticker, adjusted_date)
            
            if not stock_data.empty:
                date_diff = adjusted_date - timedelta(weeks=52)
                #change in s&p 500 price in %     
                sp500_change = get_stock_price("^GSPC", adjusted_date) / get_stock_price("^GSPC",date_diff)
                #change in stock price in %
                stock_change = get_stock_price(ticker,adjusted_date)/ get_stock_price(ticker,date_diff)
                #ratio of change_stock / change_s&p500
                ratio = stock_change / sp500_change
                if ratio is not None: 
                    return ratio
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")

In [53]:
#function to get the stock value corresponding to the date date

def get_stock_value(df, ticker) :
    lst_stock_value = []
    lst_stock_ratio = []
    for date in df.columns :
        date_str = date.strftime('%Y-%m-%d')
        lst_stock_value.append(get_stock_price(ticker, date))
        lst_stock_ratio.append(get_stock_value_ratio(ticker, date))
        
    df.loc['Stock value'] = lst_stock_value
    df.loc['Ratio ticker sp500'] = lst_stock_ratio
    return df

In [54]:

def create_df(tickers):
    df_complete = pd.DataFrame()
    for ticker in tickers:  
        df_row = pd.DataFrame()
        try: 
            # Get financial statements for the ticker ticker
            stock = yf.Ticker(ticker)
            balance_sheet = stock.balance_sheet
            income_statement = stock.financials
            cashflow_statement = stock.cashflow
            
            # add the stock value and flatten the dataframe
            df_bs = get_stock_value(balance_sheet,ticker)
            df_bs_f = flatten_data(df_bs,ticker)
            df_is_f = flatten_data(income_statement,ticker)
            df_cs_f = flatten_data(cashflow_statement,ticker)
            #concatenate
            df_row = pd.concat([df_bs_f,df_is_f,df_cs_f],axis=1)
            df_complete = pd.concat([df_complete, df_row])
    
        except Exception as e: 
            print(f"Error processing ticker {ticker}: {e}")
    return df_complete
        
     

In [55]:
tickers
df = create_df(tickers)


1 Failed download:
['AAMC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)')

1 Failed download:
['AAMC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)')


$AAMC: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)
$AAMC: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)
0
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
1
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
2
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
3
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
4
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
5
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
6
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
0
Error processing ticker 2022-12-31 00:00:00: can only concatenate 


1 Failed download:
['AAMC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2022-12-31 00:00:00 -> 2023-01-01 00:00:00)')


$AAMC: possibly delisted; No price data found  (1d 2022-12-31 00:00:00 -> 2023-01-01 00:00:00)



1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1640926800, endDate = 1641013200")

1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1641013200, endDate = 1641099600")

1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1641099600, endDate = 1641186000")

1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1641186000, endDate = 1641272400")

1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1641272400, endDate = 1641358800")

1 Failed download:
['AAMC']: YFChartError("%ticker%: Data doesn't exist for startDate = 1641358800, endDate = 1641445200")


...

0
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
1
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
2
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
3
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
4
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
5
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
6
Error processing ticker 2023-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
$AAN: possibly delisted; No price data found  (1d 2022-12-31 00:00:00 -> 2023-01-01 00:00:00)
$AAN: possibly delisted; No price data found  (1d 2023-01-01 00:00:00 -> 2023-01-02 00:00:00)



1 Failed download:
['AAN']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)')


$AAN: possibly delisted; No price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)
0
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
1
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
2
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
3
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
4
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
5
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
6
Error processing ticker 2022-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
0
Error processing ticker 2021-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
1
Error processing ticker 2021-12-31 00:00:00: can only co


1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1577768400, endDate = 1577854800")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1577854800, endDate = 1577941200")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1577941200, endDate = 1578027600")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1578027600, endDate = 1578114000")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1578114000, endDate = 1578200400")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1578200400, endDate = 1578286800")

1 Failed download:
['AAN']: YFChartError("%ticker%: Data doesn't exist for startDate = 1578286800, endDate = 1578373200")


0
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
1
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
2
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
3
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
4
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
5
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str
6
Error processing ticker 2019-12-31 00:00:00: can only concatenate str (not "datetime.timedelta") to str


In [50]:
df

Ticker,2023 Treasury Shares Number,2023 Ordinary Shares Number,2023 Share Issued,2023 Net Debt,2023 Total Debt,2023 Tangible Book Value,2023 Invested Capital,2023 Working Capital,2023 Net Tangible Assets,2023 Capital Lease Obligations,...,2020 Sale Of Business,2020 Purchase Of Business,2020 Change In Inventory,2020 Provisionand Write Offof Assets,2019 Short Term Debt Payments,2019 Net Business Purchase And Sale,2019 Sale Of Business,2019 Purchase Of Business,2019 Change In Inventory,2019 Provisionand Write Offof Assets
Ticker,,,,,,,,,,,,,,,,,,,,,
AAMC,2129973.0,2554512.0,4684485.0,NaN,900000.0,-125843000.0,NaN,NaN,-125843000.0,900000.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAME,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
AAN,6295216.0,30361434.0,36656650.0,134928000.0,696655000.0,522230000.0,880101000.0,385925000.0,522230000.0,502692000.0,...,359000.0,-14793000.0,0.0,30753000.0,NaN,NaN,2813000.0,NaN,NaN,NaN


In [83]:
stock = yf.Ticker('AAMC')
df_bs_test = stock.balance_sheet


In [77]:
lst_ratio

[None, None, None, None, None]

In [25]:
balance_sheet_aapl

,2023-09-30,2022-09-30,2021-09-30,2020-09-30,2019-09-30
Treasury Shares Number,0.0,NaN,NaN,NaN,NaN
Ordinary Shares Number,15550061000.0,15943425000.0,16426786000.0,16976763000.0,NaN
Share Issued,15550061000.0,15943425000.0,16426786000.0,16976763000.0,NaN
Net Debt,81123000000.0,96423000000.0,89779000000.0,74420000000.0,NaN
Total Debt,111088000000.0,120069000000.0,124719000000.0,112436000000.0,NaN
...,...,...,...,...,...
Other Short Term Investments,31590000000.0,24658000000.0,27699000000.0,52927000000.0,NaN
Cash And Cash Equivalents,29965000000.0,23646000000.0,34940000000.0,38016000000.0,NaN
Cash Equivalents,1606000000.0,5100000000.0,17635000000.0,20243000000.0,NaN
Cash Financial,28359000000.0,18546000000.0,17305000000.0,17773000000.0,NaN


In [49]:
ticker = 'AAPL'
stock = yf.Ticker(ticker)
balance_sheet = stock.balance_sheet
date = balance_sheet_aapl.columns[3]

for i in range(7):
    # Adjust date by i days
    adjusted_date = date + timedelta(days=i)
    stock_data = get_stock_price(ticker, adjusted_date)
    #change in s&p 500 price in %     
    date_diff = adjusted_date - timedelta(weeks=52)
    sp500_change = get_stock_price("^GSPC",adjusted_date) / get_stock_price("^GSPC",date_diff)
    #change in stock price in %
    stock_change = get_stock_price(ticker,adjusted_date)/ get_stock_price(ticker,date_diff)
    #ratio of change_stock / change_s&p500
    ratio = stock_change / sp500_change
    if ratio is not None: 
        print(f'this is the ratio {ratio}')
        print(f'this is the sp change {sp500_change}')
        print(f'this is the stock change {stock_change}')
       


this is the ratio 1.816573322620761
this is the sp change 1.1646309144563025
this is the stock change 2.1156374499007407
this is the ratio 1.8213555400437105
this is the sp change 1.1615355386791075
this is the stock change 2.115569188330848



1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-03 00:00:00 -> 2020-10-04 00:00:00)')


this is the ratio 1.7556915213382562
this is the sp change 1.1342847452407006
this is the stock change 1.991454110002422
$AAPL: possibly delisted; No price data found  (1d 2020-10-03 00:00:00 -> 2020-10-04 00:00:00)



1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)')


$AAPL: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)


...

$^GSPC: possibly delisted; No price data found  (1d 2019-10-05 00:00:00 -> 2019-10-06 00:00:00)
$^GSPC: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2020-10-03 00:00:00 -> 2020-10-04 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)



1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)')

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2019-10-05 00:00:00 -> 2019-10-06 00:00:00)')

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)')


$AAPL: possibly delisted; No price data found  (1d 2019-10-05 00:00:00 -> 2019-10-06 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)



1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)')

1 Failed download:
['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)')

1 Failed download:
['^GSPC']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)')

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)')

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)')


this is the ratio 1.7694479618581698
this is the sp change 1.1598651323670688
this is the stock change 2.0523209944972662
$AAPL: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)
$^GSPC: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)
$^GSPC: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2020-10-04 00:00:00 -> 2020-10-05 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2019-10-06 00:00:00 -> 2019-10-07 00:00:00)
this is the ratio 1.7694479618581698
this is the sp change 1.1598651323670688
this is the stock change 2.0523209944972662
this is the ratio 1.7694479618581698
this is the sp change 1.1598651323670688
this is the stock change 2.0523209944972662
this is the ratio 1.7362926255597926
this is the sp change 1.161735291571104
this is the stock change 2.017112419607463


In [107]:


ticker = tickers_total[3]
#date = yf.Ticker(ticker).balance_sheet.columns[]
value = 0
adjusted_date = date + timedelta(days=i)
stock_data = yf.download(ticker, start=adjusted_date, end=adjusted_date + timedelta(weeks=2),progress=False)
print(stock_data)
print(date)
print(ticker)
if not stock_data.empty:
    value =  stock_data['Close'].iloc[0]
else :
    return None
value, ticker, date


1 Failed download:
['AACI']: YFChartError("%ticker%: Data doesn't exist for startDate = 1601956800, endDate = 1603166400")


Empty DataFrame
Columns: [Open, High, Low, Close, Adj Close, Volume]
Index: []
2020-09-30 00:00:00
AACI


SyntaxError: 'return' outside function (3639899727.py, line 12)

In [104]:
 yf.download(ticker, start=adjusted_date, end=adjusted_date + timedelta(weeks=2),progress=False)


1 Failed download:
['AACI']: YFChartError("%ticker%: Data doesn't exist for startDate = 1601956800, endDate = 1603166400")


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,


In [ ]:
print(

In [7]:
df = df_bs_test
ticker = 'AAPL'
lst = []
for date in df.columns :
    print(lst)
    date_str = date.strftime('%Y-%m-%d')
    if get_stock_price(ticker, date):
        lst.append(get_stock_price(ticker, date))
df.loc['Stock value'] = lst 


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)')


[]
$AAPL: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

$AAPL: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2023-12-31 00:00:00 -> 2024-01-01 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2024-01-01 00:00:00 -> 2024-01-02 00:00:00)
[185.63999938964844]



[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2022-12-31 00:00:00 -> 2023-01-01 00:00:00)')
[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-01-01 00:00:00 -> 2023-01-02 00:00:00)')


...

$AAPL: possibly delisted; No price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2022-12-31 00:00:00 -> 2023-01-01 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2023-01-01 00:00:00 -> 2023-01-02 00:00:00)
$AAPL: possibly delisted; No price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)


[*********************100%%**********************]  1 of 1 completed

1 Failed download:
['AAPL']: YFPricesMissingError('$%ticker%: possibly delisted; No price data found  (1d 2023-01-02 00:00:00 -> 2023-01-03 00:00:00)')
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


[185.63999938964844, 125.06999969482422]
[185.63999938964844, 125.06999969482422, 177.57000732421875]


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed

[185.63999938964844, 125.06999969482422, 177.57000732421875, 132.69000244140625]


In [ ]:
df_complete = pd.DataFrame()
for ticker in tickers:    
    try: 
        # Get financial statements for the ticker ticker
        stock = yf.Ticker(ticker)
        balance_sheet = stock.balance_sheet
        income_statement = stock.financials
        cashflow_statement = stock.cashflow
        # add the stock value and flatten the dataframe
        df_bs = get_stock_value(balance_sheet,ticker)
        df_bs_f = flatten_data(df_bs)
        df_is_f = flatten_data(income_statement)
        df_cs_f = flatten_data(cashflow_statement)
        #concatenate
        df_row = pd.concat([df_bs_f,df_is_f,df_cs_f],axis=1)
        df
        df_complete = pd.concat([df_complete, df_row])

    except Exception as e: 
        print(f"Error processing ticker {ticker}: {e}")
        
        
        

In [ ]:
df_row

In [ ]:
df_preprocessed.columns

In [ ]:
tickers 

In [ ]:
df_data = pd.DataFrame

In [ ]:
df_data.mean()

In [ ]:
# Loop through tickers

for ticker in tickers:
    try:
        stock = yf.Ticker(ticker)
        
        # Get financial statements
        balance_sheet = stock.balance_sheet
        income_statement = stock.financials
        cashflow_statement = stock.cashflow
        
        # Ensure we have data for the past 4 years
        if len(balance_sheet.columns) >= 4 and len(income_statement.columns) >= 4 and len(cashflow_statement.columns) >= 4:
            row = {'Ticker' : ticker}
            for i in range(4):
                date = balance_sheet.columns[i]
                date_str = date.strftime('%Y-%m-%d')
                stock_price = get_stock_price(ticker, date)
                
                # Add data to the row with year-specific keys
                row[f'Date_{i+1}'] = date_str
                row[f'Stock_Price_{i+1}'] = stock_price
                
                for col in balance_sheet.index:
                    row[f'BS_{col}_{i+1}'] = balance_sheet.loc[col, date]
                for col in income_statement.index:
                    row[f'IS_{col}_{i+1}'] = income_statement.loc[col, date]
                for col in cashflow_statement.index:
                    row[f'CF_{col}_{i+1}'] = cashflow_statement.loc[col, date]
            print(row)
            # Write the row to the CSV file
            with open(csv_file, mode='a', newline='') as file:
                writer = csv.DictWriter(file, fieldnames=row.keys())
                
                if not header_written:
                    writer.writeheader()
                    header_written = True
                
                writer.writerow(row)
            
                    
    except Exception as e:
        print(f"Error processing ticker {ticker}: {e}")

print(f"Data extraction complete. Data saved to '{csv_file}'.")

In [ ]:
data = pd.read_csv(csv_file)

In [ ]:
data

In [ ]:
# Define the ticker symbol
ticker = 'AAPL'

# Get data for this ticker
stock_data = yf.download(ticker, start='2019-01-01', end='2023-12-31')

stock_data = yf.Ticker(tickers[0])

df_balance = stock_data.balance_sheet
df_income_s = stock.financials
df_cash = stock.cashflow

# flatten the data and set the ticker as index
df_balance_u = df_balance.unstack().reset_index()
df_balance_u['Ticker'] = df_balance_u.apply(lambda row: str(row['level_0'].year) + ' ' + row['level_1'], axis=1)
df_balance_u.set_index(keys = 'Ticker')
df_ordered = df_balance_u.drop(columns = ['level_0','level_1'])
df_transposed = df_ordered.set_index(keys='Ticker').T
df_transposed['Ticker'] = ticker
df_transposed.set_index(keys='Ticker')


In [ ]:
stock_data.balance_sheet

In [ ]:
df_bs = stock_data.balance_sheet
df_bs

In [ ]:
df_bs